# Демонстрация использования классов `Oscillogram` и `DataAnonymizer`

Этот ноутбук показывает, как использовать класс `Oscillogram` для загрузки и базового анализа осциллограмм COMTRADE, а также класс `DataAnonymizer` для анонимизации данных в файлах `.cfg` и `.dat`.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации файлы копируются во временную рабочую директорию (`temp_anonymizer_notebook_data`), так как процесс анонимизации изменяет и переименовывает исходные файлы. Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`. Вы можете адаптировать пути для использования собственных данных.

In [ ]:
import os
import shutil
import sys

# --- Пути к данным для тестирования ---
# Настройка sys.path для импорта модулей проекта (если еще не сделано в предыдущей ячейке)
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

# Базовый путь к данным, подготовленным для тестов
# Предполагается, что ноутбук находится в notebooks/, а tests/sample_data/ на том же уровне, что и notebooks/
base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
source_comtrade_for_anonymizer = os.path.join(base_sample_data_dir, "comtrade_files", "for_anonymizer_1")

# --- Создание временной рабочей директории для этого ноутбука ---
# Это важно, так как DataAnonymizer изменяет файлы (переименовывает)
notebook_temp_dir = "temp_anonymizer_notebook_data"
if os.path.exists(notebook_temp_dir):
    shutil.rmtree(notebook_temp_dir)
os.makedirs(notebook_temp_dir)

# --- Копирование необходимых файлов для демонстрации ---
# Файлы, которые будут анонимизированы
original_cfg_name = "to_anonymize.cfg"
original_dat_name = "to_anonymize.dat"
    
src_cfg_path = os.path.join(source_comtrade_for_anonymizer, original_cfg_name)
src_dat_path = os.path.join(source_comtrade_for_anonymizer, original_dat_name)

# Проверяем, существуют ли исходные файлы
cfg_file_to_load_anonymizer = None # Индикатор проблемы
if not os.path.exists(src_cfg_path) or not os.path.exists(src_dat_path):
    print(f"ОШИБКА: Исходные файлы для теста не найдены в {source_comtrade_for_anonymizer}")
    print("Пожалуйста, убедитесь, что скрипт tests/test_data_setup.py был запущен для создания тестовых данных.")
else:
    shutil.copy(src_cfg_path, os.path.join(notebook_temp_dir, original_cfg_name))
    shutil.copy(src_dat_path, os.path.join(notebook_temp_dir, original_dat_name))
    # Путь к CFG во временной директории
    cfg_file_to_load_anonymizer = os.path.join(notebook_temp_dir, original_cfg_name) 
    print(f"Демонстрационные файлы скопированы в: {notebook_temp_dir}")
    print("Содержимое временной директории:")
    for item in os.listdir(notebook_temp_dir):
        print(f"- {item}")

## 1. Использование `Oscillogram`

Класс `Oscillogram` из `core.oscillogram` является оберткой для данных COMTRADE. Он загружает файлы `.cfg` и `.dat` (используя `comtrade_APS.py` внутри) и предоставляет удобный доступ к метаданным и самим данным осциллограммы.

In [ ]:
import sys
import os
# Добавляем корневую директорию проекта в PYTHONPATH, чтобы работали импорты модулей проекта
# Это может потребовать корректировки в зависимости от того, откуда запускается ноутбук
# Если ноутбук в notebooks/, то ../ добавляет корень проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) # Assumes notebook is in 'notebooks' directory
if module_path not in sys.path:
    sys.path.append(module_path)

from core.oscillogram import Oscillogram
import pandas as pd # для отображения DataFrame

# Путь к CFG файлу, скопированному во временную директорию в предыдущей ячейке
# cfg_file_to_load_anonymizer был определен в предыдущей ячейке

if cfg_file_to_load_anonymizer and os.path.exists(cfg_file_to_load_anonymizer):
    try:
        osc = Oscillogram(cfg_file_to_load_anonymizer)
        print(f"Осциллограмма успешно загружена: {osc.filepath}")
        print(f"  File Hash (MD5 of DAT): {osc.file_hash}") 
        print(f"  Станция: {osc.station_name}")
        print(f"  Устройство: {osc.rec_dev_id}")
        print(f"  Частота: {osc.frequency} Гц")
        print(f"  Время начала записи: {osc.start_timestamp}")
        print(f"  Аналоговые каналы: {osc.analog_channel_ids}")
        print(f"  Дискретные каналы: {osc.status_channel_ids}")

        print("\nПервые 5 строк данных в виде DataFrame:")
        # Проверка версии pandas для корректного отображения
        if pd.__version__ > '1.0.0':
            try:
                from IPython.display import display
                display(osc.data_frame.head())
            except ImportError:
                print(osc.data_frame.head())
        else:
            print(osc.data_frame.head())

    except FileNotFoundError as e:
        print(f"Ошибка: Файл не найден. {e}")
    except RuntimeError as e:
        print(f"Ошибка во время выполнения при загрузке осциллограммы: {e}")
    except ImportError as e:
        print(f"Ошибка импорта: {e}. Убедитесь, что pandas и IPython установлены, если ошибка связана с ними.")
    except Exception as e:
        print(f"Произошла неожиданная ошибка: {e}")
else:
    print("Файл для загрузки (cfg_file_to_load_anonymizer) не найден или не был определен.")
    print("Убедитесь, что предыдущая ячейка выполнена успешно и файлы были скопированы.")

## 2. Использование `DataAnonymizer`

Класс `DataAnonymizer` из `preprocessing.anonymizer` предназначен для удаления конфиденциальной информации (имя станции, устройства, даты) из файлов COMTRADE и переименования файлов на основе хеш-суммы их `.dat` части.

In [ ]:
from preprocessing.anonymizer import DataAnonymizer

anonymizer = DataAnonymizer()

# Анонимизируем всю временную директорию, куда были скопированы файлы
# notebook_temp_dir определена в ячейке настройки
if 'notebook_temp_dir' in locals() and os.path.exists(notebook_temp_dir):
    print(f"\nЗапуск анонимизации для директории: {notebook_temp_dir}")
    anonymizer.anonymize_directory(notebook_temp_dir) 

    print("\nСодержимое директории после анонимизации:")
    new_cfg_name = None
    for item in os.listdir(notebook_temp_dir):
        print(f"- {item}")
        if item.endswith(".cfg"):
            new_cfg_name = item # Сохраняем имя для последующей загрузки

    error_log_path = os.path.join(notebook_temp_dir, "protected_files.txt")
    if os.path.exists(error_log_path):
        print("\nНайден лог ошибок 'protected_files.txt':")
        with open(error_log_path, "r", encoding="utf-8") as f:
            print(f.read())
    else:
        print("\nЛог ошибок 'protected_files.txt' не создан (ошибок не было или они не были записаны).")

    if new_cfg_name:
        print(f"\nЗагрузка анонимизированного файла: {new_cfg_name}")
        try:
            anonymized_osc_path = os.path.join(notebook_temp_dir, new_cfg_name)
            anonymized_osc = Oscillogram(anonymized_osc_path)
            print(f"  Станция (аноним.): '{anonymized_osc.station_name}'") 
            print(f"  Устройство (аноним.): '{anonymized_osc.rec_dev_id}'") 
            print(f"  Время начала записи (аноним.): {anonymized_osc.start_timestamp}") 
            print(f"  Хеш файла (должен совпадать с именем файла без .cfg): {anonymized_osc.file_hash}")
            print("\nДанные анонимизированного файла (первые 5 строк):")
            if pd.__version__ > '1.0.0':
                try:
                    from IPython.display import display
                    display(anonymized_osc.data_frame.head())
                except ImportError:
                    print(anonymized_osc.data_frame.head())
            else:
                print(anonymized_osc.data_frame.head())
        except Exception as e:
            print(f"Ошибка при загрузке анонимизированного файла: {e}")
    else:
        print("\nНе найден .cfg файл после анонимизации для демонстрации загрузки.")
else:
    print("Переменная notebook_temp_dir не определена или директория не существует. Пропустили анонимизацию.")

In [ ]:
# Очистка временной директории
try:
    # notebook_temp_dir определена в ячейке настройки
    if 'notebook_temp_dir' in locals() and os.path.exists(notebook_temp_dir):
        shutil.rmtree(notebook_temp_dir)
        print(f"\nВременная директория {notebook_temp_dir} удалена.")
    else:
        print("\nВременная директория для очистки не найдена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_temp_dir if 'notebook_temp_dir' in locals() else 'temp_anonymizer_notebook_data'}: {e}")